In [1]:
import uproot3
import numpy as np
import ROOT

Welcome to JupyROOT 6.24/06


In [2]:
# file = uproot3.open("../samples/vertexperformance_AMVF_pu10_refitted_full.root")
# file = uproot3.open("/Users/dejavu/Projects/Vertexing/ACTS/vertex_fitter/Test_fitter/vertexperformance_AMVF_pu100_refitted.root")
file = uproot3.open("../samples/vertexperformance_AMVF_pu100_refitted.root")


In [ ]:
file.keys()

In [3]:
outfile = ROOT.TFile("reso.root", "recreate")

truth_tree_arrays = file["Truth_Vertex_PV_Selected"].arrays(namedecode="utf-8")
reco_tree_arrays = file["Reco_Vertex"].arrays(namedecode="utf-8")
reco_tree_arrays_avg = file["refitted_avg"].arrays(namedecode="utf-8")
reco_tree_arrays_weighted = file["refitted_weighted_avg"].arrays(namedecode="utf-8")

In [4]:
def Get_ind_best_reco_HS_sum_pt2(reco_tree_arrays, i_event, reco_type):
    # Return the index of the best reconstructed HS via sum of pt^2 method 
    trk_pt_sq = ((1./reco_tree_arrays["reco_vtx_fitted_trk_qp"][i_event])*np.sin(reco_tree_arrays["reco_vtx_fitted_trk_theta"][i_event]) )**2
    if reco_type=="avg":
        n_vtx = len(reco_tree_arrays["refitted_avg_vtx_vz"][i_event])
    elif reco_type=="weighted":
        n_vtx = len(reco_tree_arrays["refitted_weighted_vtx_vz"][i_event])
    else:
        n_vtx = len(reco_tree_arrays["reco_vtx_vz"][i_event])
            
    vtx_trk_pt_sq_sum = np.zeros(n_vtx)
    for i in range(n_vtx):
        vtx_trk_pt_sq_sum[i] = trk_pt_sq[reco_tree_arrays["reco_vtx_fitted_trk_vtxID"][i_event] == i].sum()
    
#     print(vtx_trk_pt_sqrt_sum)
    
    return vtx_trk_pt_sq_sum.argmax()

In [5]:
def Get_local_PU_density(truth_tree_arrays, i_event, xyz_dist_window = 2.0):
    # Calculate the PU density around the truth HS vertex 
    truth_vtx_vx = truth_tree_arrays["truth_vtx_vx"][i_event]
    truth_vtx_vy = truth_tree_arrays["truth_vtx_vy"][i_event]
    truth_vtx_vz = truth_tree_arrays["truth_vtx_vz"][i_event]
    
    dist_to_truth_HS = (truth_vtx_vx - truth_vtx_vx[0])**2 +(truth_vtx_vy - truth_vtx_vy[0])**2 + (truth_vtx_vz - truth_vtx_vz[0])**2 
    n_local_truth = len(np.where(dist_to_truth_HS< 2.0**2)[0])
    return (n_local_truth - 1)/(2 * xyz_dist_window)
#     return local_PU_density 

In [6]:
canvas = ROOT.TCanvas()
hs_reco_eff = ROOT.TEfficiency("hs_reco_eff", "HS Reconstruction Efficiency; Local PU density; eff", 12, 0, 6)
hs_sel_eff = ROOT.TEfficiency("hs_sel_eff", "HS selection and Reconstruction Efficiency; Local PU density; eff", 12, 0, 6)

hs_truth_long_reso_vs_PU = ROOT.TH2D("hs_truth_long_reso_vs_PU", "HS Longitudinal Resolution vs PU density; Local PU density; [mm]", 12, 0, 6.0, 20, -0.08, 0.08);

# hs_truth_trans_reso_vs_PU = ROOT.TH2D();
# hs_trans_reso = ROOT.TH1D();
# hs_trans_bias = ROOT.TH1D();

whole_match_matrix = []
# In the future, whole_match_matrix will be used to classify 
N_evts=1000
# for i_event in range(len(reco_tree_arrays["event_id"])):
for i_event in range(N_evts):

    if i_event % 100 == 0:
        print(f"{i_event} events processed.")
    
    truth_vtx_vz = truth_tree_arrays["truth_vtx_vz"][i_event]
    reco_vtx_vz = reco_tree_arrays["reco_vtx_vz"][i_event]
    match_matrix = np.zeros((len(reco_vtx_vz),len(truth_vtx_vz)))
    # declare the truth-mathing matrix as a list, filled in the next 2 loops 
    # Can be normalized 

    for i in range(len(reco_vtx_vz)):
        for j in range(len(truth_vtx_vz)):
            test_truth = truth_tree_arrays["truth_vtx_fitted_trk_z0"][i_event][truth_tree_arrays["truth_vtx_fitted_trk_vtxID"][i_event] == j]
            test_reco = reco_tree_arrays["reco_vtx_fitted_trk_z0"][i_event][reco_tree_arrays["reco_vtx_fitted_trk_vtxID"][i_event] == i]
            match_matrix[i,j] = len(np.intersect1d(test_reco, test_truth))
            
    whole_match_matrix.append(match_matrix)
    
    
    # Find the index of best reco HS via 2 ways 
    ind_best_reco_HS_nTrk = match_matrix[:,0].argmax()
    ind_best_reco_HS_sumpt2 = Get_ind_best_reco_HS_sum_pt2(reco_tree_arrays, i_event, "reco")
    
    # dist_best_reco_to_truth_HS_sq = (reco_tree_arrays["reco_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0])**2 + (reco_tree_arrays["reco_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0])**2 + (reco_tree_arrays["reco_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0])**2 
    residual_z = reco_tree_arrays["reco_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0]
    residual_x = reco_tree_arrays["reco_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0]
    residual_y = reco_tree_arrays["reco_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0]
    
    dist_best_reco_to_truth_HS_sq =  residual_z**2 + residual_x**2 + residual_y**2
    trhth_HS_vtx_recoed = False
    if dist_best_reco_to_truth_HS_sq < (0.1 ** 2):
        trhth_HS_vtx_recoed = True

    trhth_HS_vtx_seled = (bool)(ind_best_reco_HS_nTrk == ind_best_reco_HS_sumpt2)

    
    local_PU_density = Get_local_PU_density(truth_tree_arrays, i_event)
    hs_reco_eff.Fill(trhth_HS_vtx_recoed, local_PU_density)
    hs_sel_eff.Fill(trhth_HS_vtx_recoed and trhth_HS_vtx_seled, local_PU_density)
    if trhth_HS_vtx_recoed:
        hs_truth_long_reso_vs_PU.Fill(local_PU_density, residual_z)

0 events processed.
100 events processed.
200 events processed.
300 events processed.
400 events processed.
500 events processed.
600 events processed.
700 events processed.
800 events processed.
900 events processed.


In [ ]:
hs_truth_long_reso_vs_PU.Draw()
canvas.Draw()

In [7]:
outfile.cd()
hs_truth_long_reso_vs_PU.Write()


626

In [ ]:
hs_sel_eff.Draw()
canvas.Draw()

## Refitted 

In [10]:
%%time
canvas_avg = ROOT.TCanvas()
hs_reco_eff_avg = ROOT.TEfficiency("hs_reco_eff", "HS Reconstruction Efficiency(avg); Local PU density; eff", 12, 0, 6)
hs_sel_eff_avg = ROOT.TEfficiency("hs_sel_eff", "HS selection and Reconstruction Efficiency(avg); Local PU density; eff", 12, 0, 6)

hs_truth_long_reso_vs_PU_avg = ROOT.TH2D("hs_truth_long_reso_vs_PU_avg", "HS Longitudinal Resolution vs PU density; Local PU density; [mm]", 12, 0, 6.0, 20, -0.08, 0.08);

whole_match_matrix_avg = []
# In the future, whole_match_matrix will be used to classify 

# for i_event in range(len(reco_tree_arrays_avg["event_id"])):
for i_event in range(N_evts):
    if i_event % 100 == 0:
        print(f"{i_event} events processed.")
    truth_vtx_vz = truth_tree_arrays["truth_vtx_vz"][i_event]
    reco_vtx_vz = reco_tree_arrays_avg["refitted_avg_vtx_vz"][i_event]
    match_matrix = np.zeros((len(reco_vtx_vz),len(truth_vtx_vz)))
    # declare the truth-mathing matrix as a list, filled in the next 2 loops 
    # Can be normalized 

    for i in range(len(reco_vtx_vz)):
        for j in range(len(truth_vtx_vz)):
            test_truth = truth_tree_arrays["truth_vtx_fitted_trk_z0"][i_event][truth_tree_arrays["truth_vtx_fitted_trk_vtxID"][i_event] == j]
            test_reco = reco_tree_arrays_avg["reco_vtx_fitted_trk_z0"][i_event][reco_tree_arrays_avg["reco_vtx_fitted_trk_vtxID"][i_event] == i]
            match_matrix[i,j] = len(np.intersect1d(test_reco, test_truth))
            
    whole_match_matrix.append(match_matrix)
    
    
    # Find the index of best reco HS via 2 ways 
    ind_best_reco_HS_nTrk = match_matrix[:,0].argmax()
    ind_best_reco_HS_sumpt2 = Get_ind_best_reco_HS_sum_pt2(reco_tree_arrays_avg, i_event, "avg")
    
    # dist_best_reco_to_truth_HS_sq = (reco_tree_arrays_avg["refitted_avg_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0])**2 + (reco_tree_arrays_avg["refitted_avg_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0])**2 + (reco_tree_arrays_avg["refitted_avg_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0])**2 
    residual_z = reco_tree_arrays_avg["refitted_avg_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0]
    residual_x = reco_tree_arrays_avg["refitted_avg_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0]
    residual_y = reco_tree_arrays_avg["refitted_avg_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0]
    
    dist_best_reco_to_truth_HS_sq =  residual_z**2 + residual_x**2 + residual_y**2
    trhth_HS_vtx_recoed = False
    if dist_best_reco_to_truth_HS_sq < (0.1 ** 2):
        trhth_HS_vtx_recoed = True

    trhth_HS_vtx_seled = (bool)(ind_best_reco_HS_nTrk == ind_best_reco_HS_sumpt2)

    
    local_PU_density = Get_local_PU_density(truth_tree_arrays, i_event)
    hs_reco_eff_avg.Fill(trhth_HS_vtx_recoed, local_PU_density)
    hs_sel_eff_avg.Fill(trhth_HS_vtx_recoed and trhth_HS_vtx_seled, local_PU_density)
    if trhth_HS_vtx_recoed:
        hs_truth_long_reso_vs_PU_avg.Fill(local_PU_density, residual_z)

0 events processed.
100 events processed.
200 events processed.
300 events processed.
400 events processed.
500 events processed.
600 events processed.
700 events processed.
800 events processed.
900 events processed.
CPU times: user 1min 38s, sys: 302 ms, total: 1min 38s
Wall time: 1min 38s


Warning in <TFile::Append>: Replacing existing TH1: hs_truth_long_reso_vs_PU_avg (Potential memory leak).


In [11]:
outfile.cd()
hs_truth_long_reso_vs_PU_avg.Write()

625

In [ ]:
hs_reco_eff_avg.Draw()
canvas_avg.Draw()

In [ ]:
hs_sel_eff_avg.Draw()
canvas_avg.Draw()

### Refitted Weighted 

In [12]:
canvas_weighted = ROOT.TCanvas()
hs_reco_eff_weighted = ROOT.TEfficiency("hs_reco_eff", "HS Reconstruction Efficiency(avg); Local PU density; eff", 12, 0, 6)
hs_sel_eff_weighted = ROOT.TEfficiency("hs_sel_eff", "HS selection and Reconstruction Efficiency(avg); Local PU density; eff", 12, 0, 6)
hs_truth_long_reso_vs_PU_weighted = ROOT.TH2D("hs_truth_long_reso_vs_PU_weighted", "HS Longitudinal Resolution vs PU density; Local PU density; [mm]", 12, 0, 6.0, 20, -0.08, 0.08);


whole_match_matrix_weighted = []
# In the future, whole_match_matrix will be used to classify 

# for i_event in range(len(reco_tree_arrays_weighted["event_id"])):
for i_event in range(N_evts):

    if i_event % 100 == 0:
        print(f"{i_event} events processed.")
        
    truth_vtx_vz = truth_tree_arrays["truth_vtx_vz"][i_event]
    reco_vtx_vz = reco_tree_arrays_weighted["refitted_weighted_vtx_vz"][i_event]
    match_matrix = np.zeros((len(reco_vtx_vz),len(truth_vtx_vz)))
    # declare the truth-mathing matrix as a list, filled in the next 2 loops 
    # Can be normalized 

    for i in range(len(reco_vtx_vz)):
        for j in range(len(truth_vtx_vz)):
            test_truth = truth_tree_arrays["truth_vtx_fitted_trk_z0"][i_event][truth_tree_arrays["truth_vtx_fitted_trk_vtxID"][i_event] == j]
            test_reco = reco_tree_arrays_weighted["reco_vtx_fitted_trk_z0"][i_event][reco_tree_arrays_weighted["reco_vtx_fitted_trk_vtxID"][i_event] == i]
            match_matrix[i,j] = len(np.intersect1d(test_reco, test_truth))
            
    whole_match_matrix.append(match_matrix)
    
    
    # Find the index of best reco HS via 2 ways 
    ind_best_reco_HS_nTrk = match_matrix[:,0].argmax()
    ind_best_reco_HS_sumpt2 = Get_ind_best_reco_HS_sum_pt2(reco_tree_arrays_weighted, i_event, "weighted")
    
    # dist_best_reco_to_truth_HS_sq = (reco_tree_arrays_weighted["refitted_weighted_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0])**2 + (reco_tree_arrays_weighted["refitted_weighted_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0])**2 + (reco_tree_arrays_weighted["refitted_weighted_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0])**2 
    residual_z = reco_tree_arrays_weighted["refitted_weighted_vtx_vz"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vz"][i_event][0]
    residual_x = reco_tree_arrays_weighted["refitted_weighted_vtx_vx"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vx"][i_event][0]
    residual_y = reco_tree_arrays_weighted["refitted_weighted_vtx_vy"][i_event][ind_best_reco_HS_nTrk] - truth_tree_arrays["truth_vtx_vy"][i_event][0]
    
    trhth_HS_vtx_recoed = False
    if dist_best_reco_to_truth_HS_sq < (0.1 ** 2):
        trhth_HS_vtx_recoed = True

    trhth_HS_vtx_seled = (bool)(ind_best_reco_HS_nTrk == ind_best_reco_HS_sumpt2)

    
    local_PU_density = Get_local_PU_density(truth_tree_arrays, i_event)
    hs_reco_eff_weighted.Fill(trhth_HS_vtx_recoed, local_PU_density)
    hs_sel_eff_weighted.Fill(trhth_HS_vtx_recoed and trhth_HS_vtx_seled, local_PU_density)
    if trhth_HS_vtx_recoed:
        hs_truth_long_reso_vs_PU_weighted.Fill(local_PU_density, residual_z)

0 events processed.
100 events processed.
200 events processed.
300 events processed.
400 events processed.
500 events processed.
600 events processed.
700 events processed.
800 events processed.
900 events processed.


In [13]:
outfile.cd()
hs_truth_long_reso_vs_PU_weighted.Write()

663

In [ ]:
len(reco_tree_arrays_weighted["event_id"])

In [ ]:
hs_reco_eff_weighted.Draw()
canvas_weighted.Draw()

In [ ]:
hs_sel_eff_weighted.Draw()
canvas_weighted.Draw()

In [14]:
outfile.Close()

## Compare reco & sel effi

In [ ]:
canvas_reco_eff = ROOT.TCanvas()
legend_reco_eff = ROOT.TLegend(0.1,0.2,0.4,0.4)
hs_reco_eff.Draw()

hs_reco_eff_avg.SetLineColor(2)
hs_reco_eff_avg.Draw("same")

hs_reco_eff_weighted.SetLineColor(4)
hs_reco_eff_weighted.Draw("same")

legend_reco_eff.AddEntry(hs_reco_eff, "Original Fit")
legend_reco_eff.AddEntry(hs_reco_eff_avg, "Refit Avg")
legend_reco_eff.AddEntry(hs_reco_eff_weighted, "Refit Weighted")
legend_reco_eff.Draw("same")
canvas_reco_eff.Draw()

In [ ]:
canvas_sel_eff = ROOT.TCanvas()
legend_sel_eff = ROOT.TLegend(0.1,0.2,0.4,0.4)
hs_sel_eff.Draw()

hs_sel_eff_avg.SetLineColor(2)
hs_sel_eff_avg.Draw("same")

hs_sel_eff_weighted.SetLineColor(4)
hs_sel_eff_weighted.Draw("same")

legend_sel_eff.AddEntry(hs_sel_eff, "Original Fit")
legend_sel_eff.AddEntry(hs_sel_eff_avg, "Refit Avg")
legend_sel_eff.AddEntry(hs_sel_eff_weighted, "Refit Weighted")
legend_sel_eff.Draw("same")
canvas_sel_eff.Draw()

## Merge probability

In [ ]:
whole_delta_z_reco = []
for i_event in range(len(reco_tree_arrays["event_id"])):
    list_delta_z_reco = []
    for i in range(len(reco_tree_arrays["reco_vtx_vz"][i_event])-1):
        list_delta_z_reco.append(np.diff(reco_tree_arrays["reco_vtx_vz"][i_event][i:]).tolist())
    whole_delta_z_reco.append(list_delta_z_reco)
    

In [ ]:
canvas_delta_z = ROOT.TCanvas()
TH1_delta_z = ROOT.TH1F("TH1_delta_z", "delta_z", 52, -6.5, 6.5)
for _, delta_z_reco_event in enumerate(whole_delta_z_reco):
    for _, delta_z_reco in enumerate(delta_z_reco_event):
        for i in delta_z_reco:
            TH1_delta_z.Fill(i)

In [ ]:
TH1_delta_z.Scale(1/TH1_delta_z.Integral())
TH1_delta_z.Draw()
canvas_delta_z.Draw()